# <center>  🦙 Файнтюнинг больших языковых моделей 🎛</center>


## 🧑‍🎓 В этом уроке: 
<img src='../images/finetune_vs_rag.webp' align="right" width="450" height="400">

* 🎲 Разберемся, зачем и когда использовать файнтюнинг (fine-tuning)
* [🤹‍♀️ Научимся готовить датасеты для файнтюнинга](#part2)
* [🚀 Поймем, как осуществить дообучение с помощью фрэймворка [unsloth](https://github.com/unslothai/unsloth)](#part3)
* [📦👩‍💻 Освоим техники эффективного фантюна (**Peft**, **Lora**, **QLora** и прочие неприличные аббревиатуры)](#part4)
* [📦 Произведем инференс и правильное сохранение дообученной модели ](#part4)
*  🥊 [Разберём файнтюнинг на примере 2-х датасетов:](#part6)
    *  Соберать свой
    *  или взять готовый датасет с HF [статья на Хабр](https://habr.com/ru/articles/832984/)
*  [🧸 Выводы и заключения ✅](#part6)

    

<div class="alert alert-info">

**🚀 Дообучать проще, чем кажется!`**
    
<img src='../images/lamauns.png' align="right" width="350" height="478" >

* Чтобы запустить процесс файнтюнинга на ресурсах Google Colab и не ждать вечность. <br>
* Воспользуемся специальным  фрэймворком [Unsloth](https://github.com/unslothai/unsloth), который позволяет ускорить процесс файнтюнинга от 2-х до 5 раз на потребительских видеокартах (и даже на CPU), и требует меньше памяти.
* На странице проекта есть готовые удобные ноутбуки c разбором файнтюнинга самых популярных моделей - на базе одного из них мы сделали этот ноутбук. 
* Так же фреймворк позволяет запускать файнтюн без глубоких знаний нейросетей и ML.

# Установим unsloth и зависимости.

In [1]:
%%capture
# установка Unsloth, Xformers (Flash Attention) и все другие пакеты!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

#Нам нужно проверить какая версия Torch для Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

<div class="alert alert-success">
    
Импортируем библиотеки и модель, которую будем файнтюнить - `Llama-3.1-8B`. <br>
В логах увидим, что **Unsloth** пропатчил нашу систему для двукратного ускорения процесса.

In [1]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None          # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True   # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit, # Применяем QLoRA
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


<div class="alert alert-info">
    
Используемые параметры:
* `model_name` - имя модели для файнтюна (смотрим доступные [здесь](https://github.com/unslothai/unsloth))
* `max_seq_length` - максимальная длина последовательности в токенах.
* `dtype` - Тип данных для хранения весов (зависит от модели видеокарты). Для атоматического определения ставить `None`.
* `load_in_4bit` - ключевой параметр для подгрузки 4-х битной версии модели. 

<div class="alert alert-success">
    
🔥 Чтобы не переучивать полностью всю модель - будем использовать **PEFT** с **LoRA**. Так мы будем обновлять от 1 до 10% от всех параметров!

In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<div class="alert alert-info">

Для 90% случаев, вам подойдут значения параметров по умолчанию, можно экспериментировать только с параметром `r`. Полное описание всех параметров можно найти [здесь](https://docs.unsloth.ai/basics/lora-parameters-encyclopedia).

Важные параметры LoRA:
* `r` (ранг матрицы) - самый важный параметр, определяет размер дополнительной матрицы (адаптера), которую мы будем обучать в процессе файнтюнинга (рекомендуется выбирать кратно степеням двойки - 8, 16, 32 и.т.д). Так же от этого парамаетра зависит время которое будет затрачено на файнтюн, чем больше r, тем больше время.
* `target_modules` - целевые модули, веса которых будут меняться при обучении 


# <center id="part2">  🤹‍♀️ Подготовка датасета

Для файнтюна Ламы, требуется перевести каждую запись из нашего датасета в формат `Alpaca prompt`.

И обязательно в конце промпта добавляем токен конца генерации (`EOS-token`), чтобы избежать бесконечной генерации.

In [3]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

# Функция преобразования полей датасета в alpaca prompt
def formatting_prompts_func(examples):
    instructions = examples["Instruction"]
    inputs       = examples["Input"]
    outputs      = examples["Response"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }


In [4]:
from pprint import pprint
from datasets import load_dataset

# Скачиваем подготовленный датасет с HuggingFace 
dataset = load_dataset("Ivanich/datafeeling_posts", split = "train")

# Преобразуем в alpaca_prompt с помощью нашей функции и метода map.
dataset = dataset.map(formatting_prompts_func, batched = True,)

pprint(dataset[0])

{'Input': 'Запуск канала о Data Science',
 'Instruction': 'Write a post on the following topic',
 'Response': 'Всем привет!\n'
             '\n'
             'Решил запустить свой канал. Буду рассказывать здесь про свой '
             'опыт в Data Science и лайфхаки',
 'text': 'Below is an instruction that describes a task, paired with an input '
         'that provides further context. Write a response that appropriately '
         'completes the request.\n'
         '\n'
         '### Instruction:\n'
         'Write a post on the following topic\n'
         '\n'
         '### Input:\n'
         'Запуск канала о Data Science\n'
         '\n'
         '### Response:\n'
         'Всем привет!\n'
         '\n'
         'Решил запустить свой канал. Буду рассказывать здесь про свой опыт в '
         'Data Science и лайфхаки<|end_of_text|>'}


<div class="alert alert-success">
    
Видим, что готовый промпт добавился в колонку `text`.

## Перед тем как ворваться в файнтюнинг
Посмотрим, как модель справляется с задачей до файнтюна?

In [5]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Write post about the following topic: ", # instruction
        "Как запустить gen ai стартап", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
pprint(tokenizer.batch_decode(outputs))

['<|begin_of_text|>Below is an instruction that describes a task, paired with '
 'an input that provides further context. Write a response that appropriately '
 'completes the request.\n'
 '\n'
 '### Instruction:\n'
 'Write post about the following topic: \n'
 '\n'
 '### Input:\n'
 'Как запустить gen ai стартап\n'
 '\n'
 '### Response:\n'
 '1. Запустить генетический алгоритм, который генерирует идеи стартапов.\n'
 '2. Выбрать идеи, которые кажутся вам наиболее перспективными.\n'
 '3. Выполнить анализ рынка и конкурентов для этих идей.\n'
 '4. Выполнить маркетинговый анализ и анализ потребительского поведения.\n'
 '5. Выполнить анализ бизнес-моделей и стратегий.\n'
 '6. Выполнить анализ конкурентоспособности и рисков.\n'
 '7. Выполнить анализ финансового состояния и возможностей.\n'
 '8. Вып']


<div class="alert alert-success">
    
Видим, что модель выдаёт какой-то план с пунктами, и это не очень похоже на пост для канала.

# <center id="part3">   🚀 Запуск файнтюна / Дообучение модели 
Теперь давайте воспользуемся `SFTTrainer` - supervised fine-tuning (обучение без учителя) от `Huggingface TRL` . Так нам не придется писать свою reward модель, которой необходимо оценивать ответ Llama! <br>
Дополнительную документацию можно найти здесь: [Документация TRL SFT](https://huggingface.co/docs/trl/sft_trainer). <br>
Также `unsloth` поддерживает [`DPOTrainer`](https://huggingface.co/docs/trl/dpo_trainer) от TRL!

Мы делаем 60 шагов, чтобы ускорить процесс, но вы можете установить `num_train_epochs=1` для полного запуска и отключить `max_steps=None`. 


## <center> Крутая настройка для самых продвинутых | Wandb

При желании можно подключить библиотеку [Weights&Biases](https://wandb.ai/site) - бесплатный трекер экспериментов. И файнтюннить по взрослому.

In [7]:
# import wandb

# wb_token = 'WANDB_API_KEY' # ключ c сайта Weights & Biases https://wandb.ai/site
# wandb.login(key=wb_token)
# далее раскомментируйте параметр record_to

In [11]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 2,
        warmup_steps = 10,
        #num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        # report_to="wandb", # Если используете Weights & Biases
    ),
)

max_steps is given, it will override any value given in num_train_epochs


In [14]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)

print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
12.742 GB of memory reserved.


In [12]:
# Запускаем тренировку!
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 587 | Num Epochs = 4
O^O/ \_/ \    Batch size per device = 8 | Gradient Accumulation steps = 4
\        /    Total batch size = 32 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.236400
2,2.247700
3,2.328900
4,2.165200
5,2.294000
6,2.186900
7,2.269100
8,2.214700
9,2.289900
10,2.298600


In [15]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

2669.216 seconds used for training.
44.49 minutes used for training.
Peak reserved memory = 12.742 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 86.398 %.
Peak reserved memory for training % of max memory = 0.0 %.


# <center id="part4">  Инференс 💬
Давайте запустим модель! Можете изменить инструкцию и ввод, Response оставим пустым!

Попробовать вдвое ускорить инференс можно в Colab для **Llama-3.1 8b Instruct** [здесь](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)

In [17]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Write post about the following topic: ", # instruction
        "как запустить gen ai стартап", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
pprint(tokenizer.batch_decode(outputs))

['<|begin_of_text|>Below is an instruction that describes a task, paired with '
 'an input that provides further context. Write a response that appropriately '
 'completes the request.\n'
 '\n'
 '### Instruction:\n'
 'Write post about the following topic: \n'
 '\n'
 '### Input:\n'
 'как запустить gen ai стартап\n'
 '\n'
 '### Response:\n'
 '🎤 Сегодня у меня был интересный опыт. \n'
 '\n'
 '🔝 Встретился с 2-мя ребятами, которые хотят запустить свой gen ai стартап. \n'
 '\n'
 '🏆 В целом, все было классно. У них крутой опыт, хороший тимбилдинг и '
 'интересная задумка. \n'
 '\n'
 '🤔 Однако, я все же посоветовал им не запускать стартап, а просто скинуть '
 'свою идею в нашу ML лотерею. \n'
 '\n'
 '🤔 Понятно, что это не совсем']


<div class="alert alert-success">
    
✅ После файнтюна модель по тому же запросу пишет пост, и явно угадывается стиль постов канала [Datafeeling](https://t.me/datafeeling).

 **Запуск генерации в режиме стриминга:** Вы также можете использовать `TextStreamer` для непрерывного инференса, чтобы вы могли видеть генерацию токен за токеном!

In [18]:
from transformers import TextStreamer

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Write post about the following topic: ", # instruction
        "Как стать kaggle master", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")


text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Write post about the following topic: 

### Input:
Как стать kaggle master

### Response:
🏆 Как стать kaggle master

🤔 На Kaggle сейчас много мэтчей, в которых можно скинуть лидерборд. И это прекрасно, но вот только не все так просто. 

🤫 Если ты хочешь добиться успеха, то придется много потрудиться. В противном случае ты просто не добьешься успеха. И это нормально, не судите. 

🤔 Поэтому, если ты хочешь добиться успеха, то придется много потрудиться. И это нормально, не судите. 

�


### <center> Сохранение, загрузка файнтюненных моделей
Чтобы сохранить окончательную модель в качестве адаптеров `LoRA`, используйте:
* либо `push_to_hub` от **Huggingface** для онлайн-сохранения 
* либо `save_pretrained` для локального сохранения

**Важно:** сохраняется ТОЛЬКО адаптер `LoRA`, а не полная модель. Как сохранить в 16-битном формате или `GGUF` смотрите в [этом ноутбуке](https://colab.research.google.com/drive/1Ys44kVvmeZtnICzWz0xgpRnrIOjZAuxp?usp=sharing)!

In [4]:
from getpass import getpass

hf_token = getpass(prompt="Введите ваш HuggingFaceHub API ключ")
hf_username = 'Ivanich' # Вводим свой username на HugginFace

In [23]:
model.save_pretrained("datafeeling_model") # Local saving
tokenizer.save_pretrained("datafeeling_model")

model.push_to_hub(f"{hf_username}/datafeeling_model", token = hf_token) # Online saving
tokenizer.push_to_hub(f"{hf_username}/datafeeling_model", token = hf_token) # Online saving

README.md:   0%|          | 0.00/588 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/Ivanich/datafeeling_model


<div class="alert alert-success">
    
Если перейти по ссылке, которую выдает HF после загрузки на хаб - увидим, что сохранилась не вся модель, а только веса адаптера LoRA (168 Mb). 

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "datafeeling_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Write post about the following topic", # instruction
        "Распределение долей в стартапе", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWrite post about the following topic\n\n### Input:\nРаспределение долей в стартапе\n\n### Response:\nЯ думаю, что это очень важный вопрос: как распределять доли в стартапе? И, что еще важнее, как распределять доли в стартапе, когда вы еще не знаете, каким будет стартап?  Очень много людей в стартапах, которые делают это по-старому, то есть, распределяют доли по статусу: founder, cofounder, employee, contractor, investor, etc. Это не так плохо, но, с другой стороны, я не знаю ни одного стартапа, который бы был успеш']

✅ Отлично, видим, что результат получился похожим на правду

# <center id="part6"> А что ещё из полезного?

<div class="alert alert-success">
    
* [RAFT](https://arxiv.org/abs/2403.10131) - RAG + Fine Tuning - файнтюнят модель, чтобы отбирать документы для RAG
* [Finetuning ChatGPT](https://platform.openai.com/docs/guides/fine-tuning) - OpenAI и Anthropic объявили, что можно будет файнтюнить их модели на своих данных, а потом использовать эти версии моделей по API (платно).
* [Датасет](https://huggingface.co/datasets/Ivanich/datafeeling_posts) с постами канала Datafeeling, собранный в этом [ноутбуке](https://github.com/a-milenkin/LLM_practical_course/blob/main/notebooks/M5_2_Dataset_prepare.ipynb).
* Пример файнтюнинга на готовом датасете с HuggingFace c медицинскими данными. Оформили в виде [статьи на Хабр](https://habr.com/ru/articles/832984/).

# <center id="part7"> 🧸 Выводы и заключения ✅

**Плюсы FT:**
* Более консистентные результаты генрации (например, можно передать стилистику автора)
* Уменьшение галлюцинаций (следование новым доменным знаниями)
* Можно натренировать под специфический юзкейс
* Не забывает данные (хранятся в весах модели)
* Можно передать больше данных, чем в RAG (т.к. нет ограничений контекстного окна)
* Можно использовать небольшие модели - меньше затрат на дальнейшее использование
* Приватность - данные не передаются открыто в промпте, а "зашиты" в весах (меньше возможностей для промпт-хакинга)
* Выше скорость инференса по сравнению с RAG (т.к. не нужно делать семантический поиск по базе и короче промпт, быстрее начинается генерация)

**Минусы FT:**
* Нужно собрать гораздо больше данных (самая трудоемкая и важная часть)
* Повышенный порог входа - нужны более глубокие технические знания
* Нужно больше ресурсов и времени на старте
* Необходимость переучивать при выходе новых версий модели
* Иногда все равно придется использовать RAG для доступа к новой часто изменяющейся информации